# Install necessary libs, if the computer don't have these (uncomment necessary lib)

In [4]:
# !pip install paramiko
# !pip install mysql-connector-python
# !pip install sshtunnel
# !pip install numpy
# !pip install pandas

# Load Libs

In [1]:
import paramiko
import mysql.connector
import sshtunnel
import numpy as np
import pandas as pd
import timeit

ModuleNotFoundError: No module named 'paramiko'

***
* Check for NaN function

In [ ]:
def check_nan(df):
    missing_val_count_by_column = (df.isna().sum())
    columns_with_nan = missing_val_count_by_column[missing_val_count_by_column > 0]
    print('Dados com NaN:')
    print(columns_with_nan)
    print('')
    print('columns_with_nan.shape', columns_with_nan.shape)
    print('')

* chronometer

In [ ]:
def check_time(start, end):
    seg = end - start
    dias = seg // 86400
    Seg_Restantes_Após_Dias = seg % 86400
    horas = Seg_Restantes_Após_Dias // 3600         # Divisão inteira
    Seg_Restantes_Apos_Hora = seg % 3600            # Resto da divisão por 3600
    minutos = Seg_Restantes_Apos_Hora // 60         # Divisão inteira dos segundos restantes
    Seg_Restantes_Apos_Minutos = seg % 60
    #print(minutos,'minutos','e', '%.2f' % Seg_Restantes_Apos_Minutos,'segundos')
    print(dias,'dias', horas,'horas', minutos,'minutos','e', '%.2f' % Seg_Restantes_Apos_Minutos,'segundos')

***
* Read CSV

In [ ]:
coord = pd.read_csv('../outputs/dutra_coords_querys.csv')

In [ ]:
coord[['Latitude_min', 'Latitude_max', 'Longitude_min', 'Longitude_max']]

***
* MySQL connector

In [ ]:
connection = mysql.connector.MySQLConnection(user="root",
                                             password="",
                                             host='127.0.0.1',
                                             database='coldchain',
                                             port=3306)

***
# Array to Dataframe

**datapoint**' - Cada tupla da tabela representa uma (i) leitura de eHook/tag OU (ii) uma localização aferida pelo smartphone. Caso (i), a coluna `session_id` será diferente de nulo, caso (ii) `session_id` será nulo, indicando que não a tupla não tem relação com tag, apenas com o escanner/celular.

features importantes:

'id' = identificador;

'datetime' = horario da leitura

'latitude e longitude' = localização;

'session_id' = se é ehook ou smartphone 

               ehook / tag != NULL \
               smartfone == NULL (se comporta como SLS);
             
'appinstance_id' = tabela appinstance - origem do dado


In [ ]:
df_datapoint = pd.read_sql_query('''SELECT id, datetime, latitude, longitude, geo_accuracy, session_id, appinstance_id
                                    FROM coldchain.datapoint
                                    LIMIT 10;''', connection)
df_datapoint

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
* **session** - Um tag/eHook/#tag pode ter múltiplas sessões. Cada sessão possui múltiplas leituras atreladas. É apenas uma organização das leituras em um dado contexto. No logger do coldchain é uma viagem que o logger faz, no eHook está em desuso, apesar da estrutura ser populada

'id' = identificador;



In [ ]:
df_session = pd.read_sql_query('''SELECT id, identifier, count, last_datapoint_time, tag_id
                                  FROM coldchain.session 
                                  WHERE temperature_max IS NULL 
                                  LIMIT 10;''', connection)
df_session

* **tag** - Um tag pode ser um logger ColdCshapen, tag 915MHz PA, eHook, etc. Cada eHook também possui uma tupla na tabela **tag_ehook**, que permite determinar quais tuplas da tabela tag são referentes a eHooks

In [ ]:
df_tag = pd.read_sql_query("""SELECT id, tid, epc, alias, last_datapoint_time, company_id, owner_id, place_tag_id 
                              FROM coldchain.tag
                              ORDER BY id;""", connection)
df_tag

* **tag_ehook**, que permite determinar quais tuplas da tabela tag são referentes a eHooks

In [ ]:
df_tag_ehook = pd.read_sql_query("""SELECT id, timestamp_base, tag_id, tag_group_id, tag_use_id
                                   FROM coldchain.tag_ehook    
                                   ORDER BY id
                                   LIMIT 10;""", connection)
df_tag_ehook

* Separar e extrair leituras dos eHooks por KM, por dia e período do dia

'session_id' = se é ehook ou smartphone 

               ehook / tag != NULL

In [ ]:
connection = mysql.connector.MySQLConnection(user="root",
                                             password="",
                                             host='127.0.0.1',
                                             database='coldchain',
                                             port=3306)

In [ ]:
start=timeit.default_timer()

df_ehook_km = pd.read_sql_query('''SELECT
                                       #k.id, k.km, k.state, 
                                       #d.id AS datapoint_tab_id, datetime, geo_accuracy, temperature, session_id AS datapoint_tab_session_id,
                                       #session.id AS session_tab_id, session.tag_id AS session_tab_tag_id,
                                       #tag_ehook.tag_id AS tag_ehook_tab_tag_id,
                                       #HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 AS esp_00h_06hs,
                                       #HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 AS esp_06_12hs,
                                       #HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 AS esp_12_18,
                                       #HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 AS esp_18_24hs
                                       CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 00 AND 05 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_00h_06hs,
                                       CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 06 AND 11 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_06_12hs,
                                       CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 12 AND 17 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_12_18,
                                       CAST(sum(CASE WHEN HOUR(d.datetime) BETWEEN 18 AND 23 AND MINUTE(d.datetime) BETWEEN 00 AND 59 THEN 1 ELSE 0 END) AS SIGNED) AS esp_18_24hs
                                   FROM datapoint AS d
                                       JOIN dutra_kms AS k
                                       JOIN session
                                           ON session.id = d.session_id
                                       JOIN tag_ehook
                                           ON tag_ehook.tag_id = session.tag_id
                                   #WHERE (longitude BETWEEN -46.589 AND -46.587)
                                       #AND (latitude BETWEEN -23.527 AND -23.525)
                                   WHERE d.longitude > k.long_min AND d.longitude < k.long_max
                                       AND d.latitude > k.lat_min AND d.latitude < k.lat_max
                                       AND ((datetime > '2020-03-07' - interval 2 day) AND (datetime < '2020-03-07'))
                                       AND (geo_accuracy <= 700)
                                       AND (tag_ehook.tag_id IS NOT NULL)
                                       AND (session_id IS NOT null)
                                       AND (temperature IS NULL)
                                       and k.id <= 10
                                   #order by k.km
                                   ORDER BY k.km DESC;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_ehook_km

In [ ]:
# print(df_ehook_km.to_string())

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
* **appinstance** - representa a instalação de um App, integrado à lib Android/iOS eTracker, em um aparelho celular. Um aparelho pode ter múltiplos aplicativos que implementam a eTracker. Cada tupla da tabela datapoint necessariamente tem um appinstance associado, que é de onde veio o dado

In [ ]:
start=timeit.default_timer()
df_appinstance = pd.read_sql_query("""SELECT id, position, company_id, sector_id, otacfg_id, reporter_id
                                      FROM coldchain.appinstance
                                      LIMIT 10;""", connection)
end=timeit.default_timer()
check_time(start, end)

df_appinstance

* **reporter** - o aparelho celular ou SLS que é dispositivo dedicado de leitura de eHooks / #tag

In [ ]:
df_reporter = pd.read_sql_query("""SELECT id, identifier, alias, user_agent, reporterdevice_id 
                                   FROM coldchain.reporter
                                   ORDER BY id
                                   LIMIT 10;""", connection)
df_reporter

* reporterdevice

In [ ]:
df_reporterdevice = pd.read_sql_query("""SELECT id, devicemodel_id, deviceosversion_id, libversion_id
                                         FROM coldchain.reporterdevice    
                                         ORDER BY id
                                         LIMIT 10;""", connection)
df_reporterdevice

* deviceosversion

In [ ]:
df_deviceosversion = pd.read_sql_query("""SELECT *
                                         FROM coldchain.deviceosversion
                                         ORDER BY id
                                         LIMIT 10;""", connection)
df_deviceosversion.drop(['create_by', 'create_time', 'update_by', 'update_time', 'active'], axis=1)

* Separar e extrair leituras dos reporters por KM, por dia e período do dia

'session_id' = se é ehook ou smartphone 

               smartfone == NULL (se comporta como SLS);

In [ ]:
start=timeit.default_timer()

df_rep_km = pd.read_sql_query('''SELECT 
                                       CAST(sum(HOUR(datapoint.datetime) BETWEEN 00 AND 05 AND MINUTE(datapoint.datetime) BETWEEN 00 AND 59) AS SIGNED) AS esp_00h_06hs,
                                       CAST(sum(HOUR(datapoint.datetime) BETWEEN 06 AND 11 AND MINUTE(datapoint.datetime) BETWEEN 00 AND 59) AS SIGNED) AS esp_06_12hs,
                                       CAST(sum(HOUR(datapoint.datetime) BETWEEN 12 AND 17 AND MINUTE(datapoint.datetime) BETWEEN 00 AND 59) AS SIGNED) AS esp_12_18,
                                       CAST(sum(HOUR(datapoint.datetime) BETWEEN 18 AND 23 AND MINUTE(datapoint.datetime) BETWEEN 00 AND 59) AS SIGNED) AS esp_18_24hs
                                FROM datapoint
                                    JOIN appinstance
                                        ON appinstance.id = datapoint.appinstance_id
                                    JOIN reporter 
                                        ON reporter.id = appinstance.reporter_id
                                    JOIN reporterdevice 
                                        ON reporterdevice.id = reporter.reporterdevice_id
                                    JOIN deviceosversion 
                                        ON deviceosversion.id = reporterdevice.deviceosversion_id
                                WHERE (latitude BETWEEN -23.527 AND -23.525)
                                    AND (longitude BETWEEN -46.589 AND -46.587)
                                    AND (datetime BETWEEN (last_day(datetime) - interval 3 day) AND last_day(datetime))
                                    AND (geo_accuracy <= 700)
                                    AND (session_id IS null)
                                    AND (appinstance_id IS NOT NULL)
                                    AND (temperature IS NULL)
                                    AND appinstance.company_id = 14 
                                    AND deviceosversion.release NOT LIKE 'iOS%' 
                                ORDER BY datetime DESC
                                ;''', connection)

end=timeit.default_timer()

check_time(start, end)

df_rep_km

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
* Close Connections

In [ ]:
connection.close()

print('FINISH!')